In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display, Image

In [177]:
'''
s = source
t = target
g = grass
f = forest
h = hill
m = mountain
w = wall
'''

map_1 = [['s','g','g','f','f','g','g','f'],
         ['g','g','f','h','h','f','g','g'],
         ['f','f','h','m','h','f','h','g'],
         ['f','h','m','h','f','h','m','h'],
         ['f','h','h','f','g','g','h','m'],
         ['f','f','f','g','g','g','g','h'],
         ['g','g','g','g','h','g','g','g'],
         ['g','m','m','m','m','h','g','t']]

map_2 = [['s','g','g','f','f','g','g','f'],
         ['g','g','f','h','h','f','g','g'],
         ['f','f','h','m','h','f','h','g'],
         ['f','h','m','h','f','h','m','h'],
         ['f','h','h','f','g','f','h','m'],
         ['f','f','f','g','g','g','g','h'],
         ['g','g','g','g','h','g','g','g'],
         ['g','m','m','m','m','h','g','t']]

map_3 = [['s','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','g'],
         ['g','g','g','g','g','g','g','t'],
        ]

maps = [map_1, map_2, map_3]
map_files = ['small_map1.png', 'small_map2.png', 'small_map3.png']

directions = [[-1, 0 ], # Up
             [ 0, -1],  # Left
             [ 1, 0 ],  # Down
             [ 0, 1 ]]  # Right

direction_symbols = ["^", # Up 
                     "<", # Left
                     "V", # Down
                     ">"] # Right

# def build_map(graph):
#     legend = {'g':1, 'f':2, 'h':3, 'm':4, 'w':float('inf')}
#     source = [0,0]
#     target = [len(graph)-1, len(graph[0])-1]
#     grid = [[1 for col in range(len(graph[row]))] for row in range(len(graph))]
#     for row in range(len(graph)):
#         for col in range(len(graph[0])):
#             if graph[row][col] == 's':
#                 source = [row,col]
#             elif graph[row][col] =='t':
#                 target = [row,col]
#             else:
#                 grid[row][col] = legend[graph[row][col]]
#     return source, target, grid

def print_grid(name, grid):
    print(name + ": ")
    for row in grid:
        print(row)
    print()

In [164]:
class Graph:
    
    directions = [[-1, 0 ], # Up
                       [ 0, -1],  # Left
                       [ 1, 0 ],  # Down
                       [ 0, 1 ]]  # Right
    
    cost_legend = {'g':1, 'f':2, 'h':3, 'm':4, 'w':float('inf')}
    
    def __init__(self, grid):
        self.map = grid
        self.parse_map(grid)
        
    
    def parse_map(self, grid):
        self.height = len(grid)
        self.width = len(grid[0])
        self.source = [0,0]
        self.target = [len(grid)-1, len(grid[0])-1]
        self.costs = [[1 for col in range(len(grid[row]))] for row in range(len(grid))]
        for row in range(len(self.costs)):
            for col in range(len(self.costs[0])):
                if grid[row][col] == 's':
                    self.source = [row,col]
                elif grid[row][col] =='t':
                    self.target = [row,col]
                else:
                    self.costs[row][col] = self.cost_legend[grid[row][col]]
       
    def in_bounds(self, loc):
        row, col = loc
        return 0 <= row < self.height and 0 <= col < self.width

    def neighbors(self, loc):
        row, col = loc
        neighbors = []
        for direction in self.directions:
            neighbor = [(row + direction[0]), (col + direction[1])]
            if self.in_bounds(neighbor):
                neighbors.append(neighbor)
        return neighbors
    
    def cost(self, loc):
        row, col = loc
        return self.costs[row][col]

In [170]:
def dijkstra(graph):
    Q = []
    Q.append([0, graph.source])
    cur_costs = [[float('inf') for col in range(graph.width)] for row in range(graph.height)]
    prev = [[' ' for col in range(graph.width)] for row in range(graph.height)]
    cur_costs[graph.source[0]][graph.source[1]] = 0
    steps = 0
    
    while len(Q) > 0:
        steps += 1
        # Get the closest node
        Q.sort()
        Q.reverse()
        closest = Q.pop()
        current = closest[1]
        
        if current == graph.target:
            return cur_costs, prev, steps
        
#         for neighbor in graph.neighbors(current):
#             # add the cost of moving into the grid square
#             new_cost = cur_costs[current[0]][current[1]] + graph.cost(neighbor)
#             if new_cost < cur_costs[neighbor[0]][neighbor[1]]:
#                 cur_costs[neighbor[0]][neighbor[1]] = new_cost
#                 Q.append([new_cost, neighbor])

        for i in range(len(directions)):
            n_row = current[0] + directions[i][0]
            n_col = current[1] + directions[i][1]
            if 0 <= n_row < graph.height and 0 <= n_col < graph.width:
                new_cost = cur_costs[current[0]][current[1]] + graph.cost([n_row, n_col])
                if new_cost < cur_costs[n_row][n_col]:
                    cur_costs[n_row][n_col] = new_cost
                    Q.append([new_cost, [n_row, n_col]])
                    prev[n_row][n_col] = i
                    
    return cur_costs, prev, steps

def build_path(prev, source, target):
    path = [[' ' for col in range(len(prev[row]))] for row in range(len(prev))]
    row = target[0]
    col = target[1]
    path[row][col] = '*'
    while row != source[0] or col != source[1]:
        new_row = row - directions[prev[row][col]][0]
        new_col = col - directions[prev[row][col]][1]
        path[new_row][new_col] = policy_values[prev[row][col]]
        row = new_row
        col = new_col
    return path

In [178]:
def run_dijkstra(map_select):
    graph = Graph(maps[map_select - 1])
    costs, prev, steps = dijkstra(graph)
    print_grid("Costs:", costs)
    path = build_path(prev, graph.source, graph.target)
    print_grid("Path", path),
    print("Number of steps: " + str(steps))
    display(Image(filename='./images/' + map_files[map_select-1]))
    
interact(run_dijkstra, map_select=[1,2,3])

interactive(children=(Dropdown(description=u'map_select', options=(1, 2, 3), value=1), Output()), _dom_classesâ€¦

<function __main__.run_dijkstra>

In [175]:
def heuristic(a, b):
    row_1, col_1 = a
    row_2, col_2 = b
    return abs(row_1 - row_2) + abs(col_1 - col_2)

def a_star(graph):
    Q = []
    Q.append([0, graph.source])
    cur_costs = [[float('inf') for col in range(graph.width)] for row in range(graph.height)]
    prev = [[' ' for col in range(graph.width)] for row in range(graph.height)]
    cur_costs[graph.source[0]][graph.source[1]] = 0
    steps = 0
    
    while len(Q) > 0:
        steps += 1
        # Get the closest node
        Q.sort()
        Q.reverse()
        closest = Q.pop()
        current = closest[1]
        
        if current == graph.target:
            return cur_costs, prev, steps
        
        for i in range(len(directions)):
            n_row = current[0] + directions[i][0]
            n_col = current[1] + directions[i][1]
            if 0 <= n_row < graph.height and 0 <= n_col < graph.width:
                new_cost = cur_costs[current[0]][current[1]] + graph.cost([n_row, n_col])
                if new_cost < cur_costs[n_row][n_col]:
                    cur_costs[n_row][n_col] = new_cost
                    Q.append([new_cost + heuristic(graph.target, [n_row, n_col]), [n_row, n_col]])
                    prev[n_row][n_col] = i
                    
    return cur_costs, prev, steps

In [179]:
def run_a_star(map_select):
    graph = Graph(maps[map_select - 1])
    costs, prev, steps = a_star(graph)
    print_grid("Costs:", costs)
    path = build_path(prev, graph.source, graph.target)
    print_grid("Path", path),
    print("Number of steps: " + str(steps))
    display(Image(filename='./images/' + map_files[map_select-1]))
    
interact(run_a_star, map_select=[1,2,3])

interactive(children=(Dropdown(description=u'map_select', options=(1, 2, 3), value=1), Output()), _dom_classesâ€¦

<function __main__.run_a_star>